# CMake

* cmake can take "defines" on the command line that can be used to set paths in the makefiles
    * Example:
~~~~~~~~
cmake -DCMAKE_EXE_LINKER_FLAGS_INIT="-L/Users/stephenh/projects/py-ncepbufr/src" ..
~~~~~~~~

* Other controls  
    https://cmake.org/cmake/help/v3.0/manual/cmake-variables.7.html
    
    CMAKE_ARGC  
    CMAKE_ARGV0  
    CMAKE_BINARY_DIR  
    CMAKE_COMMAND  
    ...  


# Git Flow

## When the developer doesn't have push permission to the github repository

### Developer in their local repo
~~~~~~~~
git flow init

git flow feature start <name>

make edits

git flow feature publish <name>
~~~~~~~~

### Developer on GitHub

~~~~~~~~
initiate a pull request from branch featrue/<name> to branch develop
~~~~~~~~

### Owner on GitHub

~~~~~~~~
once feature/<name> is approved, process the pull request
    merge feature/<name> into develop
    delete feature/<name>
~~~~~~~~

### Developer in their local repo

~~~~~~~~
git branch -D feature/<name>

git remote update -p  # -p prunes the remote feature/<name> branch from the local repo

git checkout master
git pull origin master

git checkout develop
git pull origin develop
~~~~~~~~

# Ecbuild

* To enable features, such as OpenMP, you use the ecbuild_add_option macro
    * The user sets an ENABLE\_ environment var (via ecbuild_add_option)
    * And the macro sets HAVE\_ accordingly
    
    * Example: OpenMP
        * User places the following in a CMakeLists.txt file

~~~~~~~~
ecbuild_add_option( FEATURE OMP
                    DEFAULT ON
                    DESCRIPTION "Use OpenMP" )
~~~~~~~~

* This macro will set the HAVE_OMP cmake variable which is then used downstream by other cmake processes

# How to resolve mege conflict with a git lfs file

```
cd ioda-bundle/iodaconv

git checkout develop
git pull

git checkout feature/mycode
git pull

git merge develop

# for each of the lfs files with conflicts
git checkout --theirs path_to_file    # choose develop version
git checkout --ours path_to_file      # choose feature/mycode version
```

# Build and test fv3-bundle on S4, 1/18/22

* When building my own jedi-stack using intel/impi, the formula for loading modules is a bit tricky. Here is what I did to get this accomplished.

```
git lfs install # I may have not needed this, but it did seem to help

module use /data/users/sherbener/projects/jedi-stack/modulefiles/apps

module load license_intal
module load git/2.30.0
module load miniconda/3.8-s4
module load jedi/intel-impi
module load odc/ecmwf-1.4.4
module load jedi-gcc/8.3

cd /data/users/sherbener/projects
git clone https://github.com/jcsda-internal/fv3-bundle
cd fv3-bundle
git clone https://github.com/jcsda-internal/jedi-cmake jedicmake
mkdir build
cd build
ecbuild --toolchain=../jedicmake/cmake/Toolchains/jcsda-S4-Intel.cmake ..
make -j4
```

# Build jedi-stack with debug settings on libraries, 1/31/22

* MPICH
    * Autoconfig
        * `--enable-g=debug`
        * `--disable-fast`
        * `--enable-debug-info`

* hdf5
    * Autoconfig
        * `--enable-build-mode=debug`
        * `--enable-optimization=debug`

* netcdf
    * Autoconfig
    * export CFLAGS, CXXFLAGS, FFLAGS, FCFLAGS
        * "-g -O0"

* pnetcdf
    * Autocoonfig
        * `--enable-debug`

* eccodes
    * cmake
    * export CFLAGS, etc.

* pio
    * cmake
    * export CFLAGS, etc.

* There are a number of packages that have `--build=production` or `--build=release` settings which can be changed to `--build=Debug`.

# fv3-bundle issue with building eckit, fckit, atlas, 2/23/22

* Tried different combinations of running ecbuild using the `BUNDLE_SKIP_<REPO>` controls which controls whether you build the repo in your bundle (BUNDLE_SKIP_ = OFF) or use the installation in modules (BUNDLE_SKIP_ = ON, which is the default).

| eckit | fckit | atlas | ecbuild status |
|-------|-------|-------|----------------|
| bundle | bundle | bundle | Fail |
| bundle | bundle | module | Pass |
| bundle | module | bundle | Fail |
| bundle | module | moduel | Pass |
| module | bundle | bundle | Pass |
| module | bundle | module | Pass |
| module | module | bundle | Pass |
| module | module | moduel | Pass |

* It appears that building eckit and atlas together in the bundle does not work. All other combinations do work.

* Used `--trace-expand` to see what's going on with a failing combination
    * Found that `ENABLE_OMP=OFF` when the atlas `atlas-import.cmake` file is being constructed
    * This causes the `atlas_OMP, atlas_OMP_Fortran, atlas_OMP_CXX` features to be recorded in the `atlas-import.cmake` file as unavailable.
    * Downstream oops is looking for these features, via a check script in eckit, that doesn't find them and throws the error

* Found that adding `-DENABLE_OMP=ON` to the ecbuild command line fixes the problem since `ENABLE_OMP=ON` when the `atlas-import.cmake` file is being constructed.

# How to debug python scripts with underlying C++ libraries, 5/12/22

* The idea is to run python in one window, then run the debugger in another window (ie another process) and attach the debugger to the python process.
    * You need to run the pyhton process where you can stop and start the script
        * Either run python interactively or in pdb (python debugger) mode

* The sequence goes like:

```
# in one window run:
python3 -m pdb my_script.py

# in another window, find the process id of the python job
# that was started above
ps aux | less

# Then attach the debugger to that process and "release" that
# process by telling the debugger to continue
lldb
(lldb) attach -pid py_proc_id
(lldb) continue

# in the debugger set a breakpoint in the C++ code
(lldb) br s -f mysource.cpp -l myline

# back in the python window tell the process to continue
(pdb) c

# After continuing the python process, you should see the lldb window
# stop on the breakpoint and you can start debugging.
```


# EWOK flow, 6/1/22

## How to test my feature branch

* load ewok environment

* clone fv3-bundle and build
    * Include my feature branch

* Change JEDI_BIN environment variable to point to my bundle build/bin directory
    * Don't change JEDI_SRC variable

* After bundle is built, in a new terminal

```
# set up modules, environment for ewok
source /work/noaa/da/jedipara/ewok/bin/ewok

# point to the applications using my feature branch
JEDI_BIN="path-to-my-bundle/build/bin"

# create an R2D2 config file (if not done yet)
# check the instruction in the ewok/README.md documentation
#
# location is: $HOME/.r2d2/config.yaml

# create experiment
# take note of experiment id at end of messaging from
# the create_experiment.py script
create_experiment.py $JEDI_SRC/ewok/experiments/<exp-id>.yaml

# Error messages
#
# Look in:
#   For most tasks: /work/noaa/da/herbener/ewok/tmp/ecflow/exp-id
#   For hofx task: /work/noaa/da/herbener/ewok/tmp/ewok/datetime-id

# note if env vars change you need to restart the server
```

# spack-stack on EC2 instance, 6/10/22

* Place a copy of the EWOK PEM file on your system and restrict access
    * `~/.aws/ewok-tutorial-20220330.pem`
    * `chmod 400 ~/.aws/ewok-tutorial-20220330.pem`

* ssh to the instance
    * `ssh -i /Users/steveherbener/.aws/ewok-tutorial-20220330.pem ec2-user@ec2-44-201-179-82.compute-1.amazonaws.com`

* On the instance:

```
# set up modules and environment
scl enable gcc-toolset-11 bash
module use /home/ec2-user/sandpit/spack-stack/envs/jedi-all-gcc11/install/modulefiles/Core
module load stack-gcc/11.2.1 stack-openmpi/4.1.3 stack-python/3.9.7
module load jedi-fv3-env/main jedi-ewok-env/main

# clone fv3-bundle and fix CMakeLists.txt files (until the
# CMakeLists.txt files get updated)
#
# In top-level CMakeLists.txt replace the
# include git_functions.cmake with:

# Use external jedi-cmake or build in bundle
if(DEFINED ENV{jedi_cmake_ROOT})
  include( $ENV{jedi_cmake_ROOT}/share/jedicmake/Functions/git_functions.cmake )
else()
  ecbuild_bundle( PROJECT jedicmake GIT "https://github.com/jcsda-internal/jedi-cmake.git" BRANCH develop UPDATE )
  include( jedicmake/cmake/Functions/git_functions.cmake )
endif()

# The first run of ecbuild will fail, then in
# fv3-jedi/test/CMakeLists.txt replace the
# include git-functions.cmake with:

if(DEFINED ENV{jedi_cmake_ROOT})
  include( $ENV{jedi_cmake_ROOT}/share/jedicmake/Functions/git_functions.cmake )
else()
  include( ${CMAKE_SOURCE_DIR}/jedicmake/cmake/Functions/git_functions.cmake )
endif()

# move the the sandpit area, clone fv3-bundle and proceed

cd sandpit/steve
git clone https://github.com/jcsda-intenal/fv3-bundle

```

# Git Submodules, 6/24/22

* .gitmodule file
    * branch specs are for navigation only

* The hash on the commit that is used for the submodule is stored in the git metadata
    * Can view this using github OR by running `git log`
* The point of the feature branches in the parent repo is to have a branch with the correct submodule commit hash
    * The submodule commit hash can be changed to test different versions of the submodule

# DDT/MAP Arm Forge, 12/13/22

* Dom's DDT tutorial
    * Orion
    * DDT memory debugger
    * `ddt --connect` # prefix for connecting to remote GUI client on Mac
    * DDT version on Orion is 22.0.2
        * MacBook has 22.0.4 which is good (the major and minor version numbers need to match)

# Intel ifx, ifort update, 2/1/23

* ifx is done and ready for beta testing
    * backward compatibility with ifort
    * Optimizations, machine code generation however are different
        * Due to differnt backends: LLVM vs Intel

* ifort is not going away soon (will probably last a couple years still)

* ifort and ifx both produce ABI compatible binaries
    * This should allow us to use ifort as a debug tool
        * Say we have switched to icx, ifx and ifx throws a mysterious compiler error. We can switch to icx, ifort and see if we get better diagnostics. Or we can switch to icx, ifort for this particular file as a workaround (if ifort completes)

* LLVM (icx, icpx and ifx) will not be available for Mac/Intel hardware
    * We should be able to use icx, ifort in this case (see above)

* New development will only go into ifx
    * This include support for offloading code execution (ie, GPU)